# Qiskit Aer: Pulse Simulator

To do:
- Intro to PulseSystemModel, using generators for transmon systems
- Doing a simulation follow-up to the previous point: set up a system, create some sort of basic `Schedule`, show how to run it. Not sure if this should be something that generates an interesting plot, or just something that shows how to run the simulator.
- Example of running something on a device, then running it on the simulator, where the `PulseSystemModel` has been generated from the backend
    - This needs to be a calibration procedure, preferably similar to Dave's notebook for ease

## Introduction

This notebook shows how to use the Aer pulse simulator, which simulates an OpenPulse `Schedule` on a quantum system at the Hamiltonian level.

In [16]:
# pulse system model object
from qiskit.providers.aer.openpulse.pulse_system_model import PulseSystemModel

# function for construct transmon device models
from qiskit.providers.aer.openpulse.transmon_model_generators import transmon_system_model

## Models of pulse devices

The physical model is stored in a `PulseSystemModel` object. This object stores all information required to specify a physical system ready for pulse control.

### Generating transmon models from user specified inputs

The function `transmon_system_model` constructs a `PulseSystemModel` for a transmon system specified in terms of individual qubit terms, and a coupling map with coupling strenghts.

In [1]:
# number of qubits and cutoff dimensions
num_transmons = 2
dim_transmons = 3

# frequencies for transmon drift terms, harmonic term and anharmonic term
transmon_freqs = [5.0, 5.1]
anharmonicity_freqs = [-0.33, -0.33]

# transmon drive strengths
drive_strengths = [0.02, 0.02]

# specify coupling as a dictionary (it says qubits 0 and 1 are coupled with a coefficient 0.02)
coupling_dict = {(0,1): 0.01}

# time 
dt = 1.

# create the model
user_defined_transmon_model, cr_idx_dict = transmon_system_model(num_transmons=num_transmons, 
                                                                 dim_transmons=dim_transmons,
                                                                 transmon_freqs=transmon_freqs,
                                                                 anharm_freqs=anharmonicity_freqs,
                                                                 drive_strengths=drive_strengths,
                                                                 coupling_dict=coupling_dict,
                                                                 dt=dt)

The returned items are: 
- `system_model`, an instance of `PulseSystemModel` consumable by the simulator representing the specified transmon system model. 
- `cr_idx_dict`, a dict storing information on the index of the channel for a given CR drive. Specifically, when two qubits are coupled in the model, two CR drive channels are created for doing CR drives in both directions, and `cr_idx_dict` keeps track of the indices for these channels. E.g. in the above we specified the coupling `(0,1)`. When performing a CR drive on qubit 0 with target 1, use u channel with index `cr_idx_dict[(0,1)]`, and when performing a CR drive on qubit 1 with target 0, use u channel with index `cr_idx_dict[(1,0)]`

### Generating transmon models from backends

Alternatively, one may wish to model from an IBM backend. In this case, it is possible to construct one from the backend.

Note: don't know what the right backend to use for this is.

In [13]:
from qiskit import IBMQ

provider = IBMQ.load_account()
provider.backends()
armonk_backend = provider.get_backend('ibmq_armonk')

In [17]:
test = PulseSystemModel.from_backend(armonk_backend)


/Users/dpuzzuoli/anaconda3/envs/QiskitDev/lib/python3.7/site-packages/qiskit/providers/models/pulsedefaults.py:166: UserWarning: `qubit_freq_est` and `meas_freq_est` now have units of Hertz(Hz) rather than gigahertz(GHz).
  warnings.warn('`qubit_freq_est` and `meas_freq_est` now have units of '


AerError: 'Oscillator-type systems are not supported.'

In [21]:
getattr(armonk_backend.configuration(), 'hamiltonian')

{'description': 'Qubits are modelled as a two level system. Qubits are coupled through resonator buses. The provided Hamiltonian has been projected into the zero excitation subspace of the resonator buses leading to an effective qubit-qubit flip-flop interaction. The qubit resonance frequencies in the Hamiltonian are the cavity dressed frequencies and not exactly what is returned by the backend defaults, which also includes the dressing due to the qubit-qubit interactions.\nWarning: Currently not all system Hamiltonian information is available to the public, missing values have been replaced with 0.\n',
 'h_latex': '\\begin{align} \\mathcal{H}/\\hbar = & \\sum_{i=0}^{0}\\left(\\frac{\\omega_{q,i}}{2} (\\mathbb{1}_i-\\sigma_i^{z})+ \\Omega_{d,i}D_i(t)\\sigma_i^{X}\\right) \\\\ \\end{align}',
 'h_str': ['_SUM[i,0,0,wq{i}/2*(I{i}-Z{i})]',
  '_SUM[i,0,0,omegad{i}*X{i}||D{i}]'],
 'osc': {},
 'qub': {'0': 2},
 'vars': {'omegad0': 0, 'wq0': 31.254351486844612}}

In [24]:
{} != {}

False

In [25]:
len({})

0

In [26]:
test = {'wow': 0}